# ইন্টেন্ট-ভিত্তিক মডেল রাউটার ফাউন্ড্রি লোকাল SDK সহ

**CPU-অপ্টিমাইজড মাল্টি-মডেল রাউটিং সিস্টেম**

এই নোটবুকটি একটি বুদ্ধিমান রাউটিং সিস্টেম প্রদর্শন করে যা ব্যবহারকারীর ইন্টেন্ট অনুযায়ী সেরা ছোট ভাষার মডেলটি স্বয়ংক্রিয়ভাবে নির্বাচন করে। এটি এমন এজ ডিপ্লয়মেন্ট পরিস্থিতির জন্য আদর্শ যেখানে আপনি দক্ষতার সাথে একাধিক বিশেষায়িত মডেল ব্যবহার করতে চান।

## 🎯 আপনি যা শিখবেন

- **ইন্টেন্ট সনাক্তকরণ**: প্রম্পটগুলোকে স্বয়ংক্রিয়ভাবে শ্রেণীবদ্ধ করুন (কোড, সারসংক্ষেপ, শ্রেণীবিভাগ, সাধারণ)
- **স্মার্ট মডেল নির্বাচন**: প্রতিটি কাজের জন্য সবচেয়ে সক্ষম মডেলে রাউট করুন
- **CPU অপ্টিমাইজেশন**: মেমরি-দক্ষ মডেল যা যেকোনো হার্ডওয়্যারে কাজ করে
- **মাল্টি-মডেল ম্যানেজমেন্ট**: `--retain true` দিয়ে একাধিক মডেল লোড রাখুন
- **প্রোডাকশন প্যাটার্ন**: পুনরায় চেষ্টা করার লজিক, ত্রুটি পরিচালনা, এবং টোকেন ট্র্যাকিং

## 📋 পরিস্থিতির সংক্ষিপ্ত বিবরণ

এই প্যাটার্নটি প্রদর্শন করে:

1. **ইন্টেন্ট সনাক্তকরণ**: প্রতিটি ব্যবহারকারীর প্রম্পট শ্রেণীবদ্ধ করুন (কোড, সারসংক্ষেপ, শ্রেণীবিভাগ, বা সাধারণ)
2. **মডেল নির্বাচন**: সক্ষমতার উপর ভিত্তি করে সবচেয়ে উপযুক্ত ছোট ভাষার মডেলটি স্বয়ংক্রিয়ভাবে নির্বাচন করুন
3. **লোকাল এক্সিকিউশন**: ফাউন্ড্রি লোকাল সার্ভিসের মাধ্যমে লোকালভাবে চলমান মডেলে রাউট করুন
4. **একীভূত ইন্টারফেস**: একক চ্যাট এন্ট্রি পয়েন্ট থেকে একাধিক বিশেষায়িত মডেলে রাউটিং

**আদর্শ**: একাধিক বিশেষায়িত মডেল সহ এজ ডিপ্লয়মেন্ট যেখানে আপনি ম্যানুয়াল মডেল নির্বাচন ছাড়াই বুদ্ধিমান অনুরোধ রাউটিং চান।

## 🔧 পূর্বশর্ত

- **Foundry Local** ইনস্টল করা এবং সার্ভিস চালু
- **Python 3.8+** pip সহ
- **8GB+ RAM** (একাধিক মডেলের জন্য 16GB+ সুপারিশ করা হয়)
- **workshop_utils** মডিউল (../samples/ এ)

## 🚀 দ্রুত শুরু

নোটবুকটি:
1. আপনার সিস্টেম মেমরি সনাক্ত করবে
2. উপযুক্ত CPU মডেল সুপারিশ করবে
3. `--retain true` দিয়ে স্বয়ংক্রিয়ভাবে মডেল লোড করবে
4. নিশ্চিত করবে যে সব মডেল প্রস্তুত আছে
5. পরীক্ষামূলক প্রম্পটগুলোকে বিশেষায়িত মডেলে রাউট করবে

**আনুমানিক সেটআপ সময়**: ৫-৭ মিনিট (মডেল লোডিং সহ)


## 📦 ধাপ ১: নির্ভরশীলতা ইনস্টল করুন

অফিশিয়াল Foundry Local SDK এবং প্রয়োজনীয় লাইব্রেরিগুলি ইনস্টল করুন:

- **foundry-local-sdk**: স্থানীয় মডেল ব্যবস্থাপনার জন্য অফিশিয়াল পাইথন SDK
- **openai**: চ্যাট সম্পন্ন করার জন্য OpenAI-সামঞ্জস্যপূর্ণ API
- **psutil**: সিস্টেম মেমোরি সনাক্তকরণ এবং পর্যবেক্ষণ


In [107]:
# Install core dependencies
!pip install -q foundry-local-sdk openai psutil

## 💻 ধাপ ২: সিস্টেম মেমোরি সনাক্তকরণ

উপলব্ধ সিস্টেম মেমোরি সনাক্ত করুন, যাতে কোন CPU মডেলগুলি কার্যকরভাবে চালানো যাবে তা নির্ধারণ করা যায়। এটি আপনার হার্ডওয়্যারের জন্য সর্বোত্তম মডেল নির্বাচন নিশ্চিত করে।


In [108]:
import psutil

# Get system memory information
total_memory_gb = psutil.virtual_memory().total / (1024**3)
available_memory_gb = psutil.virtual_memory().available / (1024**3)

print('🖥️  System Memory Information')
print('=' * 70)
print(f'Total Memory:     {total_memory_gb:.2f} GB')
print(f'Available Memory: {available_memory_gb:.2f} GB')
print()

# Recommend models based on available memory
# Using model aliases - Foundry Local will automatically select CPU variant
model_aliases = []

if total_memory_gb >= 32:
    model_aliases = ['phi-4-mini', 'phi-3.5-mini', 'qwen2.5-0.5b', 'qwen2.5-coder-0.5b']
    print('✅ High Memory System (32GB+)')
    print('   Can run 3-4 models simultaneously')
elif total_memory_gb >= 16:
    model_aliases = ['phi-4-mini', 'qwen2.5-0.5b', 'phi-3.5-mini']
    print('✅ Medium Memory System (16-32GB)')
    print('   Can run 2-3 models simultaneously')
elif total_memory_gb >= 8:
    model_aliases = ['qwen2.5-0.5b', 'phi-3.5-mini']
    print('⚠️  Lower Memory System (8-16GB)')
    print('   Recommended: 2 smaller models')
else:
    model_aliases = ['qwen2.5-0.5b']
    print('⚠️  Limited Memory System (<8GB)')
    print('   Recommended: Use only smallest model')

print()
print('📋 Recommended Model Aliases for Your System:')
for model in model_aliases:
    print(f'   • {model}')

print()
print('💡 About Model Aliases:')
print('   ✓ Use base alias (e.g., phi-4-mini, not phi-4-mini-cpu)')
print('   ✓ Foundry Local automatically selects CPU variant for your hardware')
print('   ✓ No GPU required - optimized for CPU inference')
print('   ✓ Predictable memory usage and consistent performance')
print('=' * 70)

🖥️  System Memory Information
Total Memory:     63.30 GB
Available Memory: 16.19 GB

✅ High Memory System (32GB+)
   Can run 3-4 models simultaneously

📋 Recommended Model Aliases for Your System:
   • phi-4-mini
   • phi-3.5-mini
   • qwen2.5-0.5b
   • qwen2.5-coder-0.5b

💡 About Model Aliases:
   ✓ Use base alias (e.g., phi-4-mini, not phi-4-mini-cpu)
   ✓ Foundry Local automatically selects CPU variant for your hardware
   ✓ No GPU required - optimized for CPU inference
   ✓ Predictable memory usage and consistent performance


## 🤖 ধাপ ৩: স্বয়ংক্রিয় মডেল লোডিং

এই সেলটি স্বয়ংক্রিয়ভাবে:
1. Foundry Local সার্ভিস শুরু করে (যদি এটি চালু না থাকে)
2. `--retain true` ব্যবহার করে সুপারিশকৃত মডেলগুলো লোড করে (একাধিক মডেল মেমোরিতে রাখে)
3. SDK ব্যবহার করে নিশ্চিত করে যে সব মডেল প্রস্তুত আছে

⏱️ **প্রত্যাশিত সময়**: সব মডেলের জন্য ৩-৫ মিনিট


In [109]:
import subprocess
import time
import sys
import os

# Add samples directory for workshop_utils (Foundry SDK pattern)
sys.path.append(os.path.join('..', 'samples'))

print('🚀 Automatic Model Loading with SDK Verification')
print('=' * 70)

# Use top 3 recommended models (aliases)
# Foundry will automatically load CPU variants
REQUIRED_MODELS = model_aliases[:3]
print(f'📋 Loading {len(REQUIRED_MODELS)} models: {REQUIRED_MODELS}')
print('💡 Using model aliases - Foundry will load CPU variants automatically')
print()

# Step 1: Ensure Foundry Local service is running
print('📡 Step 1: Checking Foundry Local service...')
try:
    result = subprocess.run(['foundry', 'service', 'status'], 
                          capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print('   ✅ Service is already running')
    else:
        print('   ⚙️  Starting Foundry Local service...')
        subprocess.run(['foundry', 'service', 'start'], 
                      capture_output=True, text=True, timeout=30)
        time.sleep(5)
        print('   ✅ Service started')
except Exception as e:
    print(f'   ⚠️  Could not verify service: {e}')
    print('   💡 Try manually: foundry service start')

# Step 2: Load each model with --retain true
print(f'\n🤖 Step 2: Loading models with retention...')
for i, model in enumerate(REQUIRED_MODELS, 1):
    print(f'   [{i}/{len(REQUIRED_MODELS)}] Starting {model}...')
    try:
        subprocess.Popen(['foundry', 'model', 'run', model, '--retain', 'true'],
                        stdout=subprocess.DEVNULL,
                        stderr=subprocess.DEVNULL)
        print(f'       ✅ {model} loading in background')
    except Exception as e:
        print(f'       ❌ Error starting {model}: {e}')

# Step 3: Verify models are ready
print(f'\n✅ Step 3: Verifying models (this may take 2-3 minutes)...')
print('=' * 70)

try:
    from workshop_utils import get_client
    
    ready_models = []
    max_attempts = 30
    attempt = 0
    
    while len(ready_models) < len(REQUIRED_MODELS) and attempt < max_attempts:
        attempt += 1
        print(f'\n   Attempt {attempt}/{max_attempts}...')
        
        for model in REQUIRED_MODELS:
            if model in ready_models:
                continue
                
            try:
                manager, client, model_id = get_client(model, None)
                response = client.chat.completions.create(
                    model=model_id,
                    messages=[{"role": "user", "content": "test"}],
                    max_tokens=5,
                    temperature=0
                )
                
                if response and response.choices:
                    ready_models.append(model)
                    print(f'   ✅ {model} is READY')
                    
            except Exception as e:
                error_msg = str(e).lower()
                if 'connection' in error_msg or 'timeout' in error_msg:
                    print(f'   ⏳ {model} still loading...')
                else:
                    print(f'   ⚠️  {model} error: {str(e)[:60]}...')
        
        if len(ready_models) == len(REQUIRED_MODELS):
            break
            
        if len(ready_models) < len(REQUIRED_MODELS):
            time.sleep(10)
    
    # Final status
    print('\n' + '=' * 70)
    print(f'📦 Final Status: {len(ready_models)}/{len(REQUIRED_MODELS)} models ready')
    
    for model in REQUIRED_MODELS:
        if model in ready_models:
            print(f'   ✅ {model} - READY (retained in memory)')
        else:
            print(f'   ❌ {model} - NOT READY')
    
    if len(ready_models) == len(REQUIRED_MODELS):
        print('\n🎉 All models loaded and verified!')
        print('   ✅ Ready for intent-based routing')
    else:
        print(f'\n⚠️  Some models not ready. Check: foundry model ls')
        
except ImportError as e:
    print(f'\n❌ Cannot import workshop_utils: {e}')
    print('   💡 Ensure workshop_utils.py is in ../samples/')
except Exception as e:
    print(f'\n❌ Verification error: {e}')

🚀 Automatic Model Loading with SDK Verification
📋 Loading 3 models: ['phi-4-mini', 'phi-3.5-mini', 'qwen2.5-0.5b']
💡 Using model aliases - Foundry will load CPU variants automatically

📡 Step 1: Checking Foundry Local service...
   ✅ Service is already running

🤖 Step 2: Loading models with retention...
   [1/3] Starting phi-4-mini...
       ✅ phi-4-mini loading in background
   [2/3] Starting phi-3.5-mini...
       ✅ phi-3.5-mini loading in background
   [3/3] Starting qwen2.5-0.5b...
       ✅ qwen2.5-0.5b loading in background

✅ Step 3: Verifying models (this may take 2-3 minutes)...

   Attempt 1/30...
   ⚠️  phi-4-mini error: get_client() takes 1 positional argument but 2 were given...
   ⚠️  phi-3.5-mini error: get_client() takes 1 positional argument but 2 were given...
   ⚠️  qwen2.5-0.5b error: get_client() takes 1 positional argument but 2 were given...

   Attempt 2/30...
   ⚠️  phi-4-mini error: get_client() takes 1 positional argument but 2 were given...
   ⚠️  phi-3.5-min

## 🎯 ধাপ ৪: ইন্টেন্ট ডিটেকশন এবং মডেল ক্যাটালগ কনফিগার করুন

রাউটিং সিস্টেম সেট আপ করুন:
- **ইন্টেন্ট রুলস**: প্রম্পট শ্রেণীবদ্ধ করার জন্য Regex প্যাটার্ন
- **মডেল ক্যাটালগ**: মডেলের সক্ষমতাগুলোকে ইন্টেন্ট ক্যাটাগরির সাথে মানচিত্র করে
- **প্রায়োরিটি সিস্টেম**: একাধিক মডেল মিলে গেলে কোন মডেল নির্বাচন হবে তা নির্ধারণ করে

**CPU মডেলের সুবিধা**:
- ✅ GPU প্রয়োজন নেই
- ✅ স্থিতিশীল পারফরম্যান্স
- ✅ কম বিদ্যুৎ খরচ
- ✅ পূর্বানুমানযোগ্য মেমোরি ব্যবহার


In [110]:
import re

# Model capability catalog (maps model aliases to capabilities)
# Use base aliases - Foundry Local will automatically select CPU variants
CATALOG = {
    'phi-4-mini': {
        'capabilities': ['general', 'summarize', 'reasoning'],
        'priority': 3
    },
    'qwen2.5-0.5b': {
        'capabilities': ['classification', 'fast', 'general'],
        'priority': 1
    },
    'phi-3.5-mini': {
        'capabilities': ['code', 'refactor', 'technical'],
        'priority': 2
    },
    'qwen2.5-coder-0.5b': {
        'capabilities': ['code', 'programming', 'debug'],
        'priority': 1
    }
}

# Filter to only include models recommended for this system
CATALOG = {k: v for k, v in CATALOG.items() if k in model_aliases}

print('📋 Active Model Catalog (Hardware-Optimized Aliases)')
print('=' * 70)
print('💡 Using model aliases - Foundry automatically selects CPU variants')
print()
for model, info in CATALOG.items():
    caps = ', '.join(info['capabilities'])
    print(f'   • {model}')
    print(f'     Capabilities: {caps}')
    print(f'     Priority: {info["priority"]}')
    print()

# Intent detection rules (regex pattern -> intent label)
INTENT_RULES = [
    (re.compile(r'code|refactor|function|debug|program', re.I), 'code'),
    (re.compile(r'summar|abstract|tl;?dr|brief', re.I), 'summarize'),
    (re.compile(r'classif|categor|label|sentiment', re.I), 'classification'),
    (re.compile(r'explain|teach|describe', re.I), 'general'),
]

def detect_intent(prompt: str) -> str:
    """Detect intent from prompt using regex patterns.
    
    Args:
        prompt: User input text
        
    Returns:
        Intent label: 'code', 'summarize', 'classification', or 'general'
    """
    for pattern, intent in INTENT_RULES:
        if pattern.search(prompt):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Select best model for intent based on capabilities and priority.
    
    Args:
        intent: Detected intent category
        
    Returns:
        Model alias string, or first available model if no match
    """
    candidates = [
        (alias, info['priority']) 
        for alias, info in CATALOG.items() 
        if intent in info['capabilities']
    ]
    
    if candidates:
        # Sort by priority (higher = better)
        candidates.sort(key=lambda x: x[1], reverse=True)
        return candidates[0][0]
    
    # Fallback to first available model
    return list(CATALOG.keys())[0] if CATALOG else None

print('✅ Intent detection and model selection configured')
print('=' * 70)

📋 Active Model Catalog (Hardware-Optimized Aliases)
💡 Using model aliases - Foundry automatically selects CPU variants

   • phi-4-mini
     Capabilities: general, summarize, reasoning
     Priority: 3

   • qwen2.5-0.5b
     Capabilities: classification, fast, general
     Priority: 1

   • phi-3.5-mini
     Capabilities: code, refactor, technical
     Priority: 2

   • qwen2.5-coder-0.5b
     Capabilities: code, programming, debug
     Priority: 1

✅ Intent detection and model selection configured

💡 Using model aliases - Foundry automatically selects CPU variants

   • phi-4-mini
     Capabilities: general, summarize, reasoning
     Priority: 3

   • qwen2.5-0.5b
     Capabilities: classification, fast, general
     Priority: 1

   • phi-3.5-mini
     Capabilities: code, refactor, technical
     Priority: 2

   • qwen2.5-coder-0.5b
     Capabilities: code, programming, debug
     Priority: 1

✅ Intent detection and model selection configured


## 🧪 ধাপ ৫: ইন্টেন্ট ডিটেকশন পরীক্ষা করুন

নিশ্চিত করুন যে ইন্টেন্ট ডিটেকশন সিস্টেম বিভিন্ন ধরণের প্রম্পট সঠিকভাবে শ্রেণীবদ্ধ করছে।


In [111]:
# Test intent detection with sample prompts
test_prompts = [
    'Refactor this Python function for better readability',
    'Summarize the key points of this article',
    'Classify this customer feedback as positive or negative',
    'Explain how edge AI differs from cloud AI',
    'Write a function to calculate fibonacci numbers',
    'Give me a brief overview of small language models'
]

print('🧪 Testing Intent Detection')
print('=' * 70)

for prompt in test_prompts:
    intent = detect_intent(prompt)
    model = pick_model(intent)
    print(f'\nPrompt: {prompt[:50]}...')
    print(f'   Intent: {intent:15s} → Model: {model}')

print('\n' + '=' * 70)
print('✅ Intent detection working correctly')

🧪 Testing Intent Detection

Prompt: Refactor this Python function for better readabili...
   Intent: code            → Model: phi-3.5-mini

Prompt: Summarize the key points of this article...
   Intent: summarize       → Model: phi-4-mini

Prompt: Classify this customer feedback as positive or neg...
   Intent: classification  → Model: qwen2.5-0.5b

Prompt: Explain how edge AI differs from cloud AI...
   Intent: general         → Model: phi-4-mini

Prompt: Write a function to calculate fibonacci numbers...
   Intent: code            → Model: phi-3.5-mini

Prompt: Give me a brief overview of small language models...
   Intent: summarize       → Model: phi-4-mini

✅ Intent detection working correctly


## 🚀 ধাপ ৬: রাউটিং ফাংশন বাস্তবায়ন করুন

মূল রাউটিং ফাংশন তৈরি করুন যা:
1. প্রম্পট থেকে ইচ্ছা সনাক্ত করে
2. সর্বোত্তম মডেল নির্বাচন করে
3. Foundry Local SDK এর মাধ্যমে অনুরোধ সম্পাদন করে
4. টোকেন ব্যবহার এবং ত্রুটি পর্যবেক্ষণ করে

**workshop_utils প্যাটার্ন ব্যবহার করে**:
- স্বয়ংক্রিয় পুনরায় চেষ্টা সহ এক্সপোনেনশিয়াল ব্যাকঅফ
- OpenAI-সামঞ্জস্যপূর্ণ API
- টোকেন ট্র্যাকিং এবং ত্রুটি পরিচালনা


In [112]:
import os
from workshop_utils import chat_once

# Fix RETRY_BACKOFF environment variable if it has comments
if 'RETRY_BACKOFF' in os.environ:
    retry_val = os.environ['RETRY_BACKOFF'].strip().split()[0]
    try:
        float(retry_val)
        os.environ['RETRY_BACKOFF'] = retry_val
    except ValueError:
        os.environ['RETRY_BACKOFF'] = '1.0'

def route(prompt: str, max_tokens: int = 200, temperature: float = 0.7):
    """Route prompt to appropriate model based on intent.
    
    Pipeline:
    1. Detect intent using regex patterns
    2. Select best model by capability + priority
    3. Execute via Foundry Local SDK
    
    Args:
        prompt: User input text
        max_tokens: Maximum tokens in response
        temperature: Sampling temperature (0-1)
        
    Returns:
        Dict with: intent, model, output, tokens, usage, error
    """
    intent = detect_intent(prompt)
    model_alias = pick_model(intent)
    
    if not model_alias:
        return {
            'intent': intent,
            'model': None,
            'output': '',
            'tokens': None,
            'usage': {},
            'error': 'No suitable model found'
        }
    
    try:
        # Call Foundry Local via workshop_utils
        text, usage = chat_once(
            model_alias,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        # Extract token information
        usage_info = {}
        if usage:
            usage_info['prompt_tokens'] = getattr(usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(usage, 'total_tokens', None)
        
        # Estimate if not provided
        if not usage_info.get('total_tokens'):
            est_prompt = len(prompt) // 4
            est_completion = len(text or '') // 4
            usage_info['estimated_tokens'] = est_prompt + est_completion
        
        return {
            'intent': intent,
            'model': model_alias,
            'output': (text or '').strip(),
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'error': None
        }
    
    except Exception as e:
        return {
            'intent': intent,
            'model': model_alias,
            'output': '',
            'tokens': None,
            'usage': {},
            'error': f'{type(e).__name__}: {str(e)}'
        }

print('✅ Routing function ready')
print('   Using Foundry Local SDK via workshop_utils')
print('   Token tracking: Enabled')
print('   Retry logic: Automatic with exponential backoff')

✅ Routing function ready
   Using Foundry Local SDK via workshop_utils
   Token tracking: Enabled
   Retry logic: Automatic with exponential backoff


## 🎯 ধাপ ৭: রাউটিং টেস্ট চালান

বিভিন্ন প্রম্পট দিয়ে সম্পূর্ণ রাউটিং সিস্টেম পরীক্ষা করুন যাতে প্রদর্শিত হয়:
- স্বয়ংক্রিয় ইন্টেন্ট শনাক্তকরণ
- বুদ্ধিমান মডেল নির্বাচন
- একাধিক মডেলের রাউটিং যেখানে মডেলগুলো সংরক্ষিত থাকে
- টোকেন ট্র্যাকিং এবং কার্যক্ষমতার মেট্রিক্স


In [ ]:
# Test prompts covering all intent categories
test_cases = [
    {
        'prompt': 'Refactor this Python function to make it more efficient and readable',
        'expected_intent': 'code'
    },
    {
        'prompt': 'Summarize the key benefits of using small language models at the edge',
        'expected_intent': 'summarize'
    },
    {
        'prompt': 'Classify this user feedback: The app is slow but the UI looks great',
        'expected_intent': 'classification'
    },
    {
        'prompt': 'Explain the difference between local and cloud inference',
        'expected_intent': 'general'
    },
    {
        'prompt': 'Write a Python function to calculate the Fibonacci sequence',
        'expected_intent': 'code'
    },
    {
        'prompt': 'Give me a brief overview of the Phi model family',
        'expected_intent': 'summarize'
    }
]

print('🎯 Running Intent-Based Routing Tests')
print('=' * 80)

results = []
for i, test in enumerate(test_cases, 1):
    print(f'\n[{i}/{len(test_cases)}] Testing prompt...')
    print(f'Prompt: {test["prompt"]}')
    
    result = route(test['prompt'], max_tokens=150)
    results.append(result)
    
    print(f'   Expected Intent: {test["expected_intent"]}')
    print(f'   Detected Intent: {result["intent"]} {"✅" if result["intent"] == test["expected_intent"] else "⚠️"}')
    print(f'   Selected Model:  {result["model"]}')
    
    if result['error']:
        print(f'   ❌ Error: {result["error"]}')
    else:
        output_preview = result['output'][:100] + '...' if len(result['output']) > 100 else result['output']
        print(f'   ✅ Response: {output_preview}')
        
        tokens = result.get('tokens', 0)
        if tokens:
            usage = result.get('usage', {})
            if 'estimated_tokens' in usage:
                print(f'   📊 Tokens: ~{tokens} (estimated)')
            else:
                print(f'   📊 Tokens: {tokens}')

# Summary statistics
print('\n' + '=' * 80)
print('📊 ROUTING SUMMARY')
print('=' * 80)

success_count = sum(1 for r in results if not r['error'])
total_tokens = sum(r.get('tokens', 0) or 0 for r in results if not r['error'])
intent_accuracy = sum(1 for i, r in enumerate(results) if r['intent'] == test_cases[i]['expected_intent'])

print(f'Total Prompts:        {len(results)}')
print(f'✅ Successful:         {success_count}/{len(results)}')
print(f'❌ Failed:             {len(results) - success_count}')
print(f'🎯 Intent Accuracy:    {intent_accuracy}/{len(results)} ({intent_accuracy/len(results)*100:.1f}%)')
print(f'📊 Total Tokens Used:  {total_tokens}')

# Model usage distribution
print('\n📋 Model Usage Distribution:')
model_counts = {}
for r in results:
    if r['model']:
        model_counts[r['model']] = model_counts.get(r['model'], 0) + 1

for model, count in sorted(model_counts.items(), key=lambda x: x[1], reverse=True):
    percentage = (count / len(results)) * 100
    print(f'   • {model}: {count} requests ({percentage:.1f}%)')

if success_count == len(results):
    print('\n🎉 All routing tests passed successfully!')
else:
    print(f'\n⚠️  {len(results) - success_count} test(s) failed')
    print('   Check Foundry Local service: foundry service status')
    print('   Verify models loaded: foundry model ls')

print('=' * 80)

🎯 Running Intent-Based Routing Tests

[1/6] Testing prompt...
Prompt: Refactor this Python function to make it more efficient and readable


   Expected Intent: code
   Detected Intent: code ✅
   Selected Model:  phi-3.5-mini
   ✅ Response: To refactor a Python function for efficiency and readability, I would need to see the specific funct...
   📊 Tokens: ~158 (estimated)

[2/6] Testing prompt...
Prompt: Summarize the key benefits of using small language models at the edge
   Expected Intent: summarize
   Detected Intent: summarize ✅
   Selected Model:  phi-4-mini
   ❌ Error: APIConnectionError: Connection error.

[3/6] Testing prompt...
Prompt: Classify this user feedback: The app is slow but the UI looks great
   Expected Intent: classification
   Detected Intent: classification ✅
   Selected Model:  qwen2.5-0.5b
   ❌ Error: APIConnectionError: Connection error.

[4/6] Testing prompt...
Prompt: Explain the difference between local and cloud inference
   Expected Intent: general
   Detected Intent: general ✅
   Selected Model:  phi-4-mini
   ❌ Error: APIConnectionError: Connection error.

[5/6] Testing prompt...
Prompt: Wr

## 🔧 ধাপ ৮: ইন্টারঅ্যাকটিভ টেস্টিং

রাউটিং সিস্টেমটি কার্যকরভাবে কাজ করছে কিনা তা দেখতে আপনার নিজস্ব প্রম্পটগুলি চেষ্টা করুন!


In [ ]:
# Interactive testing - modify the prompt and run this cell
custom_prompt = "Explain how model quantization reduces memory usage"

print('🎯 Interactive Routing Test')
print('=' * 80)
print(f'Your prompt: {custom_prompt}')
print()

result = route(custom_prompt, max_tokens=200)

print(f'Detected Intent: {result["intent"]}')
print(f'Selected Model:  {result["model"]}')
print()

if result['error']:
    print(f'❌ Error: {result["error"]}')
else:
    print('✅ Response:')
    print('-' * 80)
    print(result['output'])
    print('-' * 80)
    
    if result['tokens']:
        print(f'\n📊 Tokens used: {result["tokens"]}')

print('\n💡 Try different prompts to test routing behavior!')

🎯 Interactive Routing Test
Your prompt: Explain how model quantization reduces memory usage

Detected Intent: general
Selected Model:  phi-4-mini

✅ Response:
--------------------------------------------------------------------------------
Model quantization is a technique used to reduce the memory footprint of a machine learning model, particularly deep learning models. It works by converting the high-precision weights of a neural network, typically represented as 32-bit floating-point numbers, into lower-precision representations, such as 8-bit integers or even binary values.


The primary reason for quantization is to decrease the amount of memory required to store the model's parameters. Since floating-point numbers take up more space than integers, by quantizing the weights, we can significantly reduce the model's size. This reduction in size not only saves memory but also can lead to faster computation during inference, as integer operations are generally faster than floating-poi

## 📊 ধাপ ৯: কর্মক্ষমতা বিশ্লেষণ

রাউটিং সিস্টেমের কর্মক্ষমতা এবং মডেলের ব্যবহার বিশ্লেষণ করুন।


In [ ]:
import time

# Performance benchmark
benchmark_prompts = [
    'Write a hello world function',
    'Summarize: AI at the edge is powerful',
    'Classify: Good product',
    'Explain edge computing'
]

print('⚡ Performance Benchmark')
print('=' * 80)

timings = []
for prompt in benchmark_prompts:
    start = time.time()
    result = route(prompt, max_tokens=50)
    duration = time.time() - start
    timings.append(duration)
    
    print(f'\nPrompt: {prompt[:40]}...')
    print(f'   Model: {result["model"]}')
    print(f'   Time: {duration:.2f}s')
    if result.get('tokens'):
        print(f'   Tokens: {result["tokens"]}')

print('\n' + '=' * 80)
print('📊 Performance Statistics:')
print(f'   Average response time: {sum(timings)/len(timings):.2f}s')
print(f'   Fastest response:      {min(timings):.2f}s')
print(f'   Slowest response:      {max(timings):.2f}s')
print('\n💡 Note: First request may be slower due to model initialization')
print('=' * 80)

⚡ Performance Benchmark

Prompt: Write a hello world function...
   Model: phi-3.5-mini
   Time: 3.31s
   Tokens: 60

Prompt: Write a hello world function...
   Model: phi-3.5-mini
   Time: 3.31s
   Tokens: 60

Prompt: Summarize: AI at the edge is powerful...
   Model: phi-4-mini
   Time: 49.67s
   Tokens: 84

Prompt: Summarize: AI at the edge is powerful...
   Model: phi-4-mini
   Time: 49.67s
   Tokens: 84

Prompt: Classify: Good product...
   Model: qwen2.5-0.5b
   Time: 7.21s
   Tokens: 69

Prompt: Classify: Good product...
   Model: qwen2.5-0.5b
   Time: 7.21s
   Tokens: 69

Prompt: Explain edge computing...
   Model: phi-4-mini
   Time: 49.67s
   Tokens: 72

📊 Performance Statistics:
   Average response time: 27.46s
   Fastest response:      3.31s
   Slowest response:      49.67s

💡 Note: First request may be slower due to model initialization

Prompt: Explain edge computing...
   Model: phi-4-mini
   Time: 49.67s
   Tokens: 72

📊 Performance Statistics:
   Average response time:

## 🎓 মূল বিষয়বস্তু ও পরবর্তী পদক্ষেপ

### ✅ আপনি যা শিখেছেন

1. **ইন্টেন্ট-ভিত্তিক রাউটিং**: প্রম্পটগুলো স্বয়ংক্রিয়ভাবে শ্রেণিবদ্ধ করুন এবং বিশেষায়িত মডেলের দিকে রাউট করুন  
2. **মেমরি-সচেতন নির্বাচন**: সিস্টেমের উপলব্ধ RAM অনুযায়ী CPU মডেল নির্বাচন করুন  
3. **মাল্টি-মডেল রিটেনশন**: `--retain true` ব্যবহার করে একাধিক মডেল লোড রাখুন  
4. **প্রোডাকশন প্যাটার্ন**: পুনরায় চেষ্টা করার লজিক, ত্রুটি পরিচালনা এবং টোকেন ট্র্যাকিং  
5. **CPU অপ্টিমাইজেশন**: GPU ছাড়াই দক্ষতার সাথে ডিপ্লয় করুন  

### 🚀 পরীক্ষার আইডিয়া

1. **কাস্টম ইন্টেন্ট যোগ করুন**:  
   ```python
   INTENT_RULES.append(
       (re.compile(r'translate|convert', re.I), 'translation')
   )
   ```
  
2. **অতিরিক্ত মডেল লোড করুন**:  
   ```bash
   foundry model run llama-3.2-1b-cpu --retain true
   ```
  
3. **মডেল নির্বাচন টিউন করুন**:  
   - CATALOG-এ প্রায়োরিটি মান সমন্বয় করুন  
   - আরও সক্ষমতা ট্যাগ যোগ করুন  
   - ব্যাকআপ কৌশল বাস্তবায়ন করুন  

4. **পারফরম্যান্স পর্যবেক্ষণ করুন**:  
   ```python
   import psutil
   print(f"Memory: {psutil.virtual_memory().percent}%")
   ```
  

### 📚 অতিরিক্ত রিসোর্স

- **Foundry Local SDK**: https://github.com/microsoft/Foundry-Local  
- **ওয়ার্কশপ নমুনা**: ../samples/  
- **এজ AI কোর্স**: ../../Module08/  

### 💡 সেরা অনুশীলন

✅ ক্রস-প্ল্যাটফর্মের সামঞ্জস্যপূর্ণ আচরণের জন্য CPU মডেল ব্যবহার করুন  
✅ একাধিক মডেল লোড করার আগে সিস্টেম মেমরি পরীক্ষা করুন  
✅ রাউটিং পরিস্থিতির জন্য `--retain true` ব্যবহার করুন  
✅ যথাযথ ত্রুটি পরিচালনা এবং পুনরায় চেষ্টা বাস্তবায়ন করুন  
✅ খরচ/পারফরম্যান্স অপ্টিমাইজেশনের জন্য টোকেন ব্যবহার ট্র্যাক করুন  

---

**🎉 অভিনন্দন!** আপনি Foundry Local SDK ব্যবহার করে CPU-অপ্টিমাইজড মডেল দিয়ে একটি প্রোডাকশন-রেডি ইন্টেন্ট-ভিত্তিক মডেল রাউটার তৈরি করেছেন!



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতা নিশ্চিত করার চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
